In [84]:
import pandas as pd
import boto3
import os
import sagemaker
from tqdm import tqdm_notebook
import tarfile
import sys
from sagemaker.pytorch import PyTorch

In [85]:
sagemaker_session = sagemaker.Session()
sage_makerbucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [86]:
SAGEMAKER_DATA_PREFIX = 'sagemaker/real_estate_image_type'
DATA_PATH = "../data/"
DATA_BUCKET_NAME = "lbc-ft-reco"
OBJECT_NAME = "data/real_estate/image_type/real_estate_image_type.tar.gz"
TRAINING_INSTANCE = "local"#local_gpu #"ml.p3.8xlarge"
PREDICTING_INSTANCE ="local"

In [87]:
session = boto3.Session()
s3 = session.resource('s3')
bucket = s3.Bucket(DATA_BUCKET_NAME)

In [88]:
ls ../data/

In [89]:
bucket.download_file(OBJECT_NAME, DATA_PATH + "data.tar.gz")

In [5]:
inputs = sagemaker_session.upload_data(path=DATA_PATH, bucket=sage_makerbucket, key_prefix=SAGEMAKER_DATA_PREFIX)


In [78]:
# Remove tmp data downloaded 
!sudo rm -rf /tmp/*

In [ ]:
real_estate_estimator = PyTorch(
    entry_point='../script/real_estate_image_type/train.py',
    role=role,
    framework_version='1.0.0',
    train_instance_count=1,
    train_instance_type=TRAINING_INSTANCE
)

real_estate_estimator.fit(inputs)

Creating tmp34axbyyi_algo-1-58lrk_1 ... 
Attaching to tmp34axbyyi_algo-1-58lrk_1
algo-1-58lrk_1  | 2019-06-12 16:06:43,656 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
algo-1-58lrk_1  | 2019-06-12 16:06:43,659 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-58lrk_1  | 2019-06-12 16:06:43,673 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
algo-1-58lrk_1  | 2019-06-12 16:06:43,674 sagemaker_pytorch_container.training INFO     Invoking user training script.
algo-1-58lrk_1  | 2019-06-12 16:06:43,798 sagemaker-containers INFO     Module train does not provide a setup.py. 
algo-1-58lrk_1  | Generating setup.py
algo-1-58lrk_1  | 2019-06-12 16:06:43,799 sagemaker-containers INFO     Generating setup.cfg
algo-1-58lrk_1  | 2019-06-12 16:06:43,799 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-58lrk_1  | 2019-06-12 16:06:43,799 sagemaker-containers INFO     Installing

In [81]:
from sagemaker.pytorch import PyTorchModel

real_estate_predictor = real_estsate_estimator.deploy(initial_instance_count=1,
                                             instance_type=PREDICTING_INSTANCE)

algo-1-xlhwc_1  | Processing /opt/ml/code
algo-1-xlhwc_1  | Building wheels for collected packages: train
algo-1-xlhwc_1  |   Running setup.py bdist_wheel for train ... done
algo-1-xlhwc_1  |   Stored in directory: /tmp/pip-ephem-wheel-cache-pcrzdbbw/wheels/35/24/16/37574d11bf9bde50616c67372a334f94fa8356bc7164af8ca3
algo-1-xlhwc_1  | Successfully built train
algo-1-xlhwc_1  | Installing collected packages: train
algo-1-xlhwc_1  |   Found existing installation: train 1.0.0
algo-1-xlhwc_1  |     Uninstalling train-1.0.0:
algo-1-xlhwc_1  |       Successfully uninstalled train-1.0.0
algo-1-xlhwc_1  | Successfully installed train-1.0.0
algo-1-xlhwc_1  | You are using pip version 18.1, however version 19.1.1 is available.
algo-1-xlhwc_1  | You should consider upgrading via the 'pip install --upgrade pip' command.


Exception in thread Thread-11:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 534, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 594, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 539, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpsqd2fcr9/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited wi

algo-1-xlhwc_1  | [2019-06-13 06:58:44 +0000] [69] [ERROR] Error handling request /ping
algo-1-xlhwc_1  | Traceback (most recent call last):
algo-1-xlhwc_1  |   File "/usr/local/lib/python3.6/dist-packages/sagemaker_containers/_functions.py", line 85, in wrapper
algo-1-xlhwc_1  |     return fn(*args, **kwargs)
algo-1-xlhwc_1  |   File "/usr/local/lib/python3.6/dist-packages/train.py", line 186, in model_fn
algo-1-xlhwc_1  |     model.load_state_dict(torch.load(f))
algo-1-xlhwc_1  |   File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 769, in load_state_dict
algo-1-xlhwc_1  |     self.__class__.__name__, "\n\t".join(error_msgs)))
algo-1-xlhwc_1  | RuntimeError: Error(s) in loading state_dict for DenseNet:
algo-1-xlhwc_1  | 	size mismatch for classifier.f2.weight: copying a param with shape torch.Size([1, 256]) from checkpoint, the shape in current model is torch.Size([5, 256]).
algo-1-xlhwc_1  | 	size mismatch for classifier.f2.bias: copying a param with shap

RuntimeError: Giving up, endpoint didn't launch correctly

In [ ]:
ls /opt/ml/metadata/